In [1]:
import pandas as pd

In [4]:
df = pd.read_csv("reine Daten nach Box-Verfahren.csv")
df.drop(['Unnamed: 0'], axis=1,inplace=True)
df.drop(['Torque [%]','Boring threshold [m]','Casing Length [m]','Crowd Depth [m]','Gear Mode Main Winch []','Inclination X [Grad]','Inclination Y [Grad]','Rope Force Aux. Winch. [t]','Status Rig [ ]','Torque Steps [K=1;M=2;G=3]'], axis=1,inplace=True)
df.head()

,Depth [m],Rope Force Main Winch [t],Speed of Rotary Drive [U/min],Pressure Pump 1 [bar],Pressure Pump 2 [bar],Pressure Pump 3 [bar],Pressure Pump 4 [bar],Crowd-Force [t],Torque [kNm],Main Winch Rope Speed [cm/min],time,activity from avtivitydaten
0,-0.482,15.60,0.0,3.2,147.2,192.0,160.0,18.26,0.0,23.776,2019-10-14 07:24:22,Nebenprozesszeit - Leeren
1,-0.284,15.54,0.0,3.4,147.2,200.8,221.0,18.26,0.0,22.370,2019-10-14 07:24:23,Nebenprozesszeit - Leeren
2,-0.106,15.08,0.0,3.6,100.8,179.0,188.4,15.14,0.0,20.230,2019-10-14 07:24:24,Nebenprozesszeit - Leeren
3,0.000,14.68,0.0,3.6,92.8,153.4,179.6,0.70,0.0,15.402,2019-10-14 07:24:25,Nebenprozesszeit - Leeren
4,-0.002,14.58,0.0,3.8,76.2,121.0,162.6,-11.26,0.0,17.040,2019-10-14 07:24:26,Nebenprozesszeit - Leeren


In [5]:
df['activity from avtivitydaten'].unique( )

array(['Nebenprozesszeit - Leeren', 'Hauptzeit - Ablassen',
       'Hauptzeit - Verrohrung eindrehen', 'Hauptzeit - Ziehen',
       'Hauptzeit - Bohren'], dtype=object)

In [6]:
Class = df.iloc[:,11].unique()
Class_dict = dict(zip(Class, range(len(Class))))
df['Aktivität mit Nr'] = df.iloc[:,11].apply(lambda x: Class_dict[x])

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

lb = LabelBinarizer()
lb.fit(list(Class_dict.values()))
transformed_labels = lb.transform(df['Aktivität mit Nr'])
y_bin_labels = []  # 对多分类进行0-1编码的变量
for i in range(transformed_labels.shape[1]):
    y_bin_labels.append('y' + str(i))
    df['y' + str(i)] = transformed_labels[:, i]
    
df[y_bin_labels]

,y0,y1,y2,y3,y4
0,1,0,0,0,0
1,1,0,0,0,0
2,1,0,0,0,0
3,1,0,0,0,0
4,1,0,0,0,0
...,...,...,...,...,...
112709,0,0,0,0,1
112710,0,0,0,0,1
112711,0,0,0,0,1
112712,0,0,0,0,1


In [7]:
# Feature festlegen
X = df.iloc[:,0:10]
# Ziel (y) festlegen


In [8]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, df[y_bin_labels], test_size = 0.2, random_state = 42)

# Feature Scaling, da die Einheit von jeder Feature miteinander nicht gleich ist.
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [12]:
from sklearn.tree import DecisionTreeClassifier
dt_model = DecisionTreeClassifier(criterion = 'entropy')
dt_model.fit(X_train,y_train)


DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [33]:
from sklearn.model_selection import cross_val_score
score = cross_val_score(dt_model, X_train, y_train, cv=10,scoring='accuracy')
score

array([0.93280106, 0.92569591, 0.92691583, 0.92713763, 0.93035378,
       0.93146279, 0.93512255, 0.93101919, 0.93268271, 0.92802484])

In [32]:
# durchschnittliche Genauigkeit vom den Model "Decision Tree" 
score.mean()    

0.930665059539959

### prediction the X_test

In [17]:
predict_results = dt_model.predict(X_test)
print('predict_results:', predict_results)
print('target_test:', y_test)

predict_results: [[0 0 1 0 0]
 [0 0 0 0 1]
 [0 0 1 0 0]
 ...
 [1 0 0 0 0]
 [0 0 0 1 0]
 [0 0 1 0 0]]
target_test:         y0  y1  y2  y3  y4
45497    0   0   1   0   0
100687   0   0   0   0   1
60163    0   0   1   0   0
92192    0   0   0   1   0
92136    0   0   0   1   0
...     ..  ..  ..  ..  ..
31078    0   1   0   0   0
48615    0   0   1   0   0
6757     1   0   0   0   0
81303    0   0   0   1   0
68952    0   0   1   0   0

[22543 rows x 5 columns]


In [19]:
from sklearn.metrics import accuracy_score

print(accuracy_score(predict_results, y_test))

0.9320409883334073


In [28]:
# Confusion Matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(
    y_test.values.argmax(axis=1), predict_results.argmax(axis=1))

array([[4665,  173,  145,   39,   69],
       [ 172, 2229,   89,   93,   61],
       [ 147,   65, 6288,   45,   12],
       [  31,   96,   53, 4237,   55],
       [  74,   52,   11,   48, 3594]], dtype=int64)